In [ ]:
# !pip install optuna

# 需求
使用大部分标签缺失的数据集'保险数据_半监督学习.csv'，训练一个分类器，实现类别标签的预测

# 数据预处理

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 数据预处理计算函数
def preprocessing(df):
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OrdinalEncoder
    from sklearn.compose import ColumnTransformer
    
    cat_cols= df.select_dtypes(include=["object"])   # 分类型变量
    num_cols= df.select_dtypes(include=["int", "float"])   # 数值型变量

    # 连续型数据
    num_imp= SimpleImputer(strategy='mean')  # 缺失值
    num_std= StandardScaler()  # 标准化
    num_pipeline= Pipeline(steps=[("num_imp", num_imp), ("num_std", num_std)])
    # 分类型数据
    cat_imp= SimpleImputer(strategy="most_frequent")  # 缺失值
    cat_encode= OrdinalEncoder()   # 数据编码
    cat_pipeline= Pipeline(steps=[("cat_imp", cat_imp), ("cat_encode", cat_encode)])

    col_trans= ColumnTransformer(transformers=[("num_pipeline", num_pipeline, num_cols.columns),
                                           ("cat_pipeline", cat_pipeline, cat_cols.columns),])
    # 数据集处理的计算
    transfer= col_trans.fit(df)
    return transfer

In [3]:
# 读入数据
raw_data= pd.read_csv('./data/保险数据_半监督学习.csv')
labels= raw_data.pop("resp_flag")  # y

In [4]:
# 只有少部分样本有标签
print("缺失值/总样本："+str(labels.isnull().sum())+"/"+str(len(labels)))
# print("类别比例："+str(labels.value_counts()))

缺失值/总样本：30000/39173


sklearn中的半监督学习算法要求，所有缺失的标签必须用-1填充

In [5]:
# 用-1填充标签
labels_fill= labels.fillna(-1)

# 特征数据处理
transfer= preprocessing(raw_data)
data_trans= transfer.transform(raw_data)

data_concat= pd.concat([labels_fill, pd.DataFrame(data_trans)], axis= 1)

# 保存一部分有标签样本作为测试集
mask_labeled= (labels_fill != -1)
mask_unlabeled= (labels_fill == -1)

data_labeled= data_concat[mask_labeled]
data_unlabeled= data_concat[mask_unlabeled]

# 切分测试集
from sklearn.model_selection  import train_test_split
train, test= train_test_split(data_labeled, test_size=0.2, stratify= data_labeled["resp_flag"], random_state= 42)

Xtrain= pd.concat([train, data_unlabeled])
Ytrain= Xtrain.pop("resp_flag")

# Semi supervised learning（半监督学习）
## Self Training
自训练（Self-training）:

自训练是一种简单的半监督学习方法，它首先使用已标记的数据训练一个监督学习模型。然后，该模型用于预测未标记数据的标签。预测最自信的标签可以被选择添加到训练集中，然后模型在新的、更大的训练集上重新训练。

In [23]:
# Base Classifier
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(oob_score=True)

# Self Training
from sklearn.semi_supervised import SelfTrainingClassifier
RF_self_training= SelfTrainingClassifier(RF)
RF_self_training.fit(Xtrain, Ytrain)

# 测试集模型评估
Xtest= test
Ytest= Xtest.pop("resp_flag")

from sklearn.metrics import roc_auc_score
print("AUC: ", roc_auc_score(Ytest, RF_self_training.predict_proba(Xtest)[:, 1]))

SelfTrainingClassifier(base_estimator=RandomForestClassifier(oob_score=True))

AUC:  0.8883257485029942


## Label Propagation 
### Label Propagation

In [28]:
RF_self_training.predict_proba(Xtest)[:, 1]

array([0.09, 0.15, 0.05, ..., 0.02, 0.96, 0.92])

In [29]:
# Label Propagation
from sklearn.semi_supervised import LabelPropagation

label_propagation = LabelPropagation(kernel="knn")
label_propagation.fit(Xtrain, Ytrain)

Ytrain_propagation= label_propagation.transduction_

LabelPropagation(kernel='knn')

In [30]:
Ytrain_propagation

array([0., 0., 1., ..., 1., 1., 1.])

In [31]:
from sklearn.ensemble import RandomForestClassifier
RF_propagation= RandomForestClassifier(oob_score=True)
RF_propagation.fit(Xtrain, Ytrain_propagation)

print("AUC: ", roc_auc_score(Ytest, RF_propagation.predict_proba(Xtest)[:, 1]))

RandomForestClassifier(oob_score=True)

AUC:  0.8719227544910179


In [32]:
RF_propagation.predict_proba(Xtest)

array([[0.7 , 0.3 ],
       [0.65, 0.35],
       [0.96, 0.04],
       ...,
       [0.99, 0.01],
       [0.28, 0.72],
       [0.61, 0.39]])

### Label Spreading

In [33]:
# Label Spreading
from sklearn.semi_supervised import LabelSpreading

label_spreading = LabelSpreading(kernel="knn", alpha= 0.2)
label_spreading.fit(Xtrain, Ytrain)

Ytrain_spreading= label_spreading.transduction_

LabelSpreading(kernel='knn')

In [34]:
Ytrain_spreading

array([0., 0., 1., ..., 0., 1., 1.])

In [35]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
RF_spreading= RandomForestClassifier(oob_score=True)
RF_spreading.fit(Xtrain, Ytrain_spreading)

print("AUC: ", roc_auc_score(Ytest, RF_spreading.predict_proba(Xtest)[:, 1]))

RandomForestClassifier(oob_score=True)

AUC:  0.8787538922155689


选定Label Spreading方法，尝试更多模型

In [36]:
# XGBoost
from xgboost.sklearn import XGBClassifier
XGB = XGBClassifier()
XGB.fit(Xtrain, Ytrain_spreading)

print("AUC: ", roc_auc_score(Ytest, XGB.predict_proba(Xtest)[:, 1]))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

AUC:  0.8914311377245508


In [37]:
# LightGBM
import lightgbm

train= lightgbm.Dataset(Xtrain, label= Ytrain_spreading)
test= lightgbm.Dataset(Xtest, label= Ytest)

lgb = lightgbm.train(params={}, train_set= train, valid_sets= test)

print("AUC: ", roc_auc_score(Ytest, lgb.predict(Xtest)))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002298 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2127
[LightGBM] [Info] Number of data points in the train set: 37338, number of used features: 49
[LightGBM] [Info] Start training from score 0.526515
AUC:  0.8994946107784432


# Tuning the hyper-parameters（调参）

## Grid Search（网格搜索）

In [39]:
from sklearn.model_selection import GridSearchCV
param = {"n_estimators": np.arange(100,150,20),    # 启，止，步长
         "max_depth": np.arange(3,6,2),
        "subsample": [0.8, 0.9],
        "colsample_bytree": [0.8, 0.9]}

XGB = XGBClassifier()
grid_XGB = GridSearchCV(XGB, param, cv=3, n_jobs=-1)  
grid_XGB.fit(Xtrain, Ytrain_spreading)

print("AUC: ", roc_auc_score(Ytest, grid_XGB.predict_proba(Xtest)[:, 1]))
grid_XGB.best_params_ #最优参数

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9],
                         'max_depth': array([3, 4, 5]),
                         'n_estimators': array([100, 110, 120, 130, 140]),
                         'subsample': [0.8, 0.9]})

AUC:  0.8931916167664672


{'colsample_bytree': 0.9,
 'max_depth': 5,
 'n_estimators': 120,
 'subsample': 0.9}

## Halving Search

In [40]:
# Halving 搜索
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

param = {"n_estimators": np.arange(100,150,1),    # 启，止，步长
         "max_depth": np.arange(3,6,1),
        "subsample": np.arange(0.5,1,0.1),
        "colsample_bytree": np.arange(0.5,1,0.1)}

XGB = XGBClassifier()
Hal_XGB = HalvingGridSearchCV(XGB, param, factor=3, cv=3, verbose=1, n_jobs=-1)  
Hal_XGB.fit(Xtrain, Ytrain_spreading)

print("AUC: ", roc_auc_score(Ytest, Hal_XGB.predict_proba(Xtest)[:, 1]))
Hal_XGB.best_params_ #最优参数

n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 17
max_resources_: 37338
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 3750
n_resources: 17
Fitting 3 folds for each of 3750 candidates, totalling 11250 fits
----------
iter: 1
n_candidates: 1250
n_resources: 51
Fitting 3 folds for each of 1250 candidates, totalling 3750 fits
----------
iter: 2
n_candidates: 417
n_resources: 153
Fitting 3 folds for each of 417 candidates, totalling 1251 fits
----------
iter: 3
n_candidates: 139
n_resources: 459
Fitting 3 folds for each of 139 candidates, totalling 417 fits
----------
iter: 4
n_candidates: 47
n_resources: 1377
Fitting 3 folds for each of 47 candidates, totalling 141 fits
----------
iter: 5
n_candidates: 16
n_resources: 4131
Fitting 3 folds for each of 16 candidates, totalling 48 fits
----------
iter: 6
n_candidates: 6
n_resources: 12393
Fitting 3 folds for each of 6 candidates, totalling 18 fits
----------
iter: 7
n_candidates: 2

HalvingGridSearchCV(cv=3,
                    estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rat...
                    param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                'max_depth': array([3, 4, 5]),
                                'n_estimators': array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]),
                                'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])},
                    verbose=1)

AUC:  0.9001964071856288


{'colsample_bytree': 0.7999999999999999,
 'max_depth': 3,
 'n_estimators': 108,
 'subsample': 0.7}

## Randomized Search（随机搜索）

In [41]:
# 随机搜索
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param = dict(n_estimators= np.arange(100,150,1),    # 启，止，步长
         max_depth= np.arange(3,6,1),
         subsample= uniform(loc=0.5, scale=0.5),   # 启，增加量
         colsample_bytree= uniform(loc=0.5, scale=0.5))

XGB = XGBClassifier()
Rand_XGB = RandomizedSearchCV(XGB, param, cv=3, n_iter=50, verbose=1, n_jobs=-1)  
Rand_XGB.fit(Xtrain, Ytrain_spreading)

print("AUC: ", roc_auc_score(Ytest, Rand_XGB.predict_proba(Xtest)[:, 1]))
Rand_XGB.best_params_ #最优参数

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'max_depth': array([3, 4, 5]),
                                        'n_estimators': array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]),
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000269924D1B50>},
                   verbose=1)

AUC:  0.8946634730538923


{'colsample_bytree': 0.6502885285360598,
 'max_depth': 5,
 'n_estimators': 141,
 'subsample': 0.9380730375335327}

In [42]:
# 查看最优模型
best_model = Rand_XGB.best_estimator_

## Bayesian Optimization（贝叶斯搜索）

In [43]:
# 贝叶斯搜索
import optuna

def objective(trial):
    param = {
        'lambda_l2': trial.suggest_float('lambda_l2', 0.01, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
    }
    
    model = lightgbm.train(params= param, train_set= train, valid_sets= test) # 建模
    AUC= roc_auc_score(Ytest, model.predict(Xtest))   # matric
    return AUC

lgb_opt = optuna.create_study(direction='maximize')
lgb_opt.optimize(objective, n_trials=100)  # 100次迭代

# 打印结果
lgb_opt.best_params
lgb_opt.best_value

[I 2024-05-19 17:13:53,876] A new study created in memory with name: no-name-6db991b9-af27-4c12-8191-7670c3c6996b


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002603 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2127
[LightGBM] [Info] Number of data points in the train set: 37338, number of used features: 49
[LightGBM] [Info] Start training from score 0.526515


[I 2024-05-19 17:13:54,420] Trial 0 finished with value: 0.8976574850299401 and parameters: {'lambda_l2': 1.3232105324442147, 'num_leaves': 109, 'feature_fraction': 0.5073613631447758, 'bagging_fraction': 0.5761853706701012}. Best is trial 0 with value: 0.8976574850299401.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2127
[LightGBM] [Info] Number of data points in the train set: 37338, number of used features: 49
[LightGBM] [Info] Start training from score 0.526515


[I 2024-05-19 17:13:54,827] Trial 1 finished with value: 0.8986359281437125 and parameters: {'lambda_l2': 5.014173649656192, 'num_leaves': 91, 'feature_fraction': 0.8856409218111932, 'bagging_fraction': 0.4820728208916143}. Best is trial 1 with value: 0.8986359281437125.


...

[I 2024-05-19 17:14:29,063] Trial 97 finished with value: 0.9025568862275448 and parameters: {'lambda_l2': 5.558759137907634, 'num_leaves': 41, 'feature_fraction': 0.7101922253264326, 'bagging_fraction': 0.6059955994339427}. Best is trial 45 with value: 0.9057269461077845.
[I 2024-05-19 17:14:29,285] Trial 98 finished with value: 0.9019604790419163 and parameters: {'lambda_l2': 4.683560623206253, 'num_leaves': 25, 'feature_fraction': 0.6850558009748444, 'bagging_fraction': 0.7257465874829259}. Best is trial 45 with value: 0.9057269461077845.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005627 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2127
[LightGBM] [Info] Number of data points in the train set: 37338, number of used features: 49
[LightGBM] [Info] Start training from score 0.526515
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001973 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2127
[LightGBM] [Info] Number of data points in the train set: 37338, number of used features: 49
[LightGBM] [Info] Start training from score 0.526515


[I 2024-05-19 17:14:29,600] Trial 99 finished with value: 0.9019628742514969 and parameters: {'lambda_l2': 4.415772892909616, 'num_leaves': 48, 'feature_fraction': 0.6097241314823667, 'bagging_fraction': 0.7090633218061947}. Best is trial 45 with value: 0.9057269461077845.


{'lambda_l2': 4.768530754185674,
 'num_leaves': 34,
 'feature_fraction': 0.8205156123700014,
 'bagging_fraction': 0.6303534743529644}

0.9057269461077845

# Manifold Learning（t-SNE）

当我们需要挖掘数据中潜在的低维/局部结构，可以考虑使用Manifold Learning方法进行降维，再训练分类器。大多数情况下我们选用t-SNE

In [45]:
# t-SNE的计算复杂度很高，非常耗时
%time
from sklearn.manifold import TSNE
Xembedded = TSNE(n_components=10, learning_rate='auto', init='random', perplexity=30, method= "exact", n_jobs=-1)
X_tsne= Xembedded.fit_transform(Xtrain)

CPU times: total: 0 ns
Wall time: 0 ns


MemoryError: Unable to allocate 5.19 GiB for an array with shape (697044453,) and data type float64